# Data Profiling for all data features

In [8]:
import pandas as pd

df = pd.read_csv("https://datasocibmproject.s3.ap-southeast-2.amazonaws.com/structured_data/capital_project_schedules_and_budgets_1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8185 entries, 0 to 8184
Data columns (total 16 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Unnamed: 0                                                  8185 non-null   int64  
 1   project_geographic_district                                 8185 non-null   int64  
 2   project_building_identifier                                 8185 non-null   object 
 3   project_school_name                                         8185 non-null   object 
 4   project_type                                                8185 non-null   object 
 5   project_description                                         8185 non-null   object 
 6   project_phase_name                                          8184 non-null   object 
 7   project_status_name                                         8185 non-null   object 
 8 

# Checking all wrongly formating not in XX/XX/XXXX or PNS or 

In [101]:
def checkDataFormatting(df, col):
    wrong_format_count = 0
    unique_value_wrong_format_dict = {}

    for date in df[col]:
        correct_format = checkLenghtData(date)
        if correct_format == False:
            # input directionary 
            unique_value_wrong_format_dict = uniqueWronglyFormatedDatas(unique_value_wrong_format_dict, date)
            wrong_format_count += 1
            
    return wrong_format_count, unique_value_wrong_format_dict

def checkLenghtData(data):
    correct_format = False
    # convert string
    data = str(data)
    # XX/XX/XXXX means month/day/year or other acceptable labels 
    if len(data) == 10 or data == 'PNS' or data == 'F&E' or data == 'CM':
        correct_format = True
        
        
    return correct_format

def uniqueWronglyFormatedDatas(dictionary, date):
    if date in dictionary:
        dictionary[date] += 1 
    else:
        dictionary[date] = 1
    return dictionary

def 

In [102]:
# is the date consistent? in the form XX / XX / XXXX

wrong_actual_date_count_start = 0
wrong_actual_unique_dict_start = {}

wrong_actual_date_count_end = 0
wrong_actual_unique_dict_end = {}

wrong_planned_date_count_end = 0
wrong_planned_unique_dict_end = {}


wrong_actual_date_count_start, wrong_actual_unique_dict_start = checkDataFormatting(df, 'project_phase_actual_start_date')
wrong_actual_date_count_end, wrong_actual_unique_dict_end = checkDataFormatting(df,'project_phase_actual_end_date')
wrong_planned_date_count_end, wrong_planned_unique_dict_end = checkDataFormatting(df, 'project_phase_planned_end_date')

print(f"For project_phase_actual_start_date there is total wrongly formated {wrong_actual_date_count_start} and unique wrong values includes {wrong_actual_unique_dict_start} \n")
print(f"For project_phase_actual_end_date there is total wrongly formated {wrong_actual_date_count_end} and unique wrong values includes {wrong_actual_unique_dict_end} \n")
print(f"For project_phase_planned_end_date there is total wrongly formated {wrong_planned_date_count_end} and unique wrong values includes {wrong_planned_unique_dict_end} \n")



For project_phase_actual_start_date there is total wrongly formated 1 and unique wrong values includes {'FTK': 1} 

For project_phase_actual_end_date there is total wrongly formated 2159 and unique wrong values includes {nan: 2158, 'FTK': 1} 

For project_phase_planned_end_date there is total wrongly formated 1324 and unique wrong values includes {'DOES': 318, 'IEH': 652, 'DOEL': 113, 'DOER': 170, 'FTK': 61, 'EMER': 4, 'DOEP': 1, '/  /': 5} 

